<a href="https://colab.research.google.com/github/meechoi38/Okta-Query/blob/main/Demo_OpenAI_Okta_Query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install langchain openai

In [ ]:
import json
import openai


openai.api_key = OPENAI_API_KEY

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd "/content/drive/MyDrive/Colab Notebooks/Okta"
%run Utility_Okta_Query.ipynb

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/Okta


/usr/local/lib/python3.10/dist-packages/langchain/__init__.py:24: UserWarning: Importing PromptTemplate from langchain root module is no longer supported.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/__init__.py:24: UserWarning: Importing OpenAI from langchain root module is no longer supported.
  warnings.warn(


In [ ]:
from langchain.agents import create_pandas_dataframe_agent
from langchain.llms import OpenAI
import pandas as pd
import re

import os
os.environ["OPENAI_API_KEY"] = "XXXXXXXX"

In [ ]:
user_query = "Who logged in on Sep 8, 2023 ?"

In [ ]:
# obtain initial response - [LATER] should render context_api_result as an output
result_api, result_REST_verb = obtain_response_from_central_agent(user_query)

df_response = connect_to_okta(result_api,result_REST_verb)
display(df_response)

 /api/v1/users
 GET


,id,status,created,activated,statusChanged,lastLogin,lastUpdated,passwordChanged,type.id,profile.firstName,profile.lastName,profile.mobilePhone,profile.secondEmail,profile.login,profile.email,credentials.provider.type,credentials.provider.name,_links.self.href
0,00u7rduj8bj6eHhfM697,PROVISIONED,2023-09-19T20:51:10.000Z,2023-09-19T20:51:11.000Z,2023-09-19T20:51:11.000Z,None,2023-09-19T20:51:11.000Z,None,oty7bhg4lp0ZO4Rn3697,Mee,Choi,123-456-7890,None,meechoi@jinga-hi.com,meechoi@jinga-hi.com,OKTA,OKTA,https://trial-5217769.okta.com/api/v1/users/00...
1,00u7bhg4qac8BksGT697,ACTIVE,2023-09-07T19:07:25.000Z,2023-09-07T19:07:26.000Z,2023-09-07T19:08:24.000Z,2023-09-08T20:36:28.000Z,2023-09-07T19:08:24.000Z,2023-09-07T19:08:24.000Z,oty7bhg4lp0ZO4Rn3697,Markus,De Shon,None,None,mdeshon@jinga-hi.com,mdeshon@jinga-hi.com,OKTA,OKTA,https://trial-5217769.okta.com/api/v1/users/00...


In [ ]:
# analyzing panda dataframe and extract python code using langchain agent

agent = create_pandas_dataframe_agent(OpenAI(temperature=0,
                                            # model_name = 'gpt-3.5-turbo'),
                                            model_name = 'gpt-4'),
                                      df_response,
                                      return_intermediate_steps=True, # this was inserted to extract intermediate steps
                                      verbose=False
                                      )

/usr/local/lib/python3.10/dist-packages/langchain/llms/openai.py:202: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/llms/openai.py:790: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [ ]:
#initial user_query

In [ ]:
user_query_1 = "Who logged in since Sep 1, 2023 ?"

In [ ]:
response = agent({"input":user_query_1})

In [ ]:
deliver_final_answer(user_query_1, agent, df_response)

print response
response
extracted_code
df['lastLogin'].dtype
df[df['lastLogin'] > '2023-09-01']

yes, it starts with df- displaying
df[df['lastLogin'] > '2023-09-01']
modified_line 
display(df[df['lastLogin'] > '2023-09-01'])



,id,status,created,activated,statusChanged,lastLogin,lastUpdated,passwordChanged,type.id,profile.firstName,profile.lastName,profile.mobilePhone,profile.secondEmail,profile.login,profile.email,credentials.provider.type,credentials.provider.name,_links.self.href
1,00u7bhg4qac8BksGT697,ACTIVE,2023-09-07T19:07:25.000Z,2023-09-07T19:07:26.000Z,2023-09-07T19:08:24.000Z,2023-09-08 20:36:28+00:00,2023-09-07T19:08:24.000Z,2023-09-07T19:08:24.000Z,oty7bhg4lp0ZO4Rn3697,Markus,De Shon,None,None,mdeshon@jinga-hi.com,mdeshon@jinga-hi.com,OKTA,OKTA,https://trial-5217769.okta.com/api/v1/users/00...


In [ ]:
# try out different questions

In [ ]:
user_query_2 = "Which user has the last name that starts with C ?"

deliver_final_answer(user_query_2, agent)

print response
response
extracted_code
df[df['profile.lastName'].str.startswith('C')]

yes, it starts with df- displaying
df[df['profile.lastName'].str.startswith('C')]
modified_line 
display(df[df['profile.lastName'].str.startswith('C')])



,id,status,created,activated,statusChanged,lastLogin,lastUpdated,passwordChanged,type.id,profile.firstName,profile.lastName,profile.mobilePhone,profile.secondEmail,profile.login,profile.email,credentials.provider.type,credentials.provider.name,_links.self.href
0,00u7rduj8bj6eHhfM697,PROVISIONED,2023-09-19T20:51:10.000Z,2023-09-19T20:51:11.000Z,2023-09-19T20:51:11.000Z,NaT,2023-09-19T20:51:11.000Z,None,oty7bhg4lp0ZO4Rn3697,Mee,Choi,123-456-7890,None,meechoi@jinga-hi.com,meechoi@jinga-hi.com,OKTA,OKTA,https://trial-5217769.okta.com/api/v1/users/00...


In [ ]:
user_query_3 = "Which user tried to change password? "

deliver_final_answer(user_query_3, agent)

print response
response
extracted_code
df[df['passwordChanged'].notna()]['profile.firstName']

yes, it starts with df- displaying
df[df['passwordChanged'].notna()]['profile.firstName']
modified_line 
display(df[df['passwordChanged'].notna()]['profile.firstName'])



1    Markus
Name: profile.firstName, dtype: object

In [ ]:
user_query_4 = "List the users who have never logged in. " \
 "For each of those users, print their full name."
deliver_final_answer(user_query_4, agent)

print response
response
extracted_code
df[df['lastLogin'].isna()][['profile.firstName', 'profile.lastName']]

yes, it starts with df- displaying
df[df['lastLogin'].isna()][['profile.firstName', 'profile.lastName']]
modified_line 
display(df[df['lastLogin'].isna()][['profile.firstName', 'profile.lastName']])



,profile.firstName,profile.lastName
0,Mee,Choi


In [ ]:
user_query_5 = "List the users who have never logged in. " \
  "Add a column showing how many days it has been since their user was created." \
  "For timestamps you may need to add a tzinfo argument showing that it's UTC time."
deliver_final_answer(user_query_5, agent)

<string>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


print response
response
extracted_code
df['created'] = pd.to_datetime(df['created'])
df_no_login = df[df['lastLogin'].isna()]
df_no_login['days_since_created'] = (pd.Timestamp.now(tz='UTC') - df_no_login['created']).dt.days
df_no_login[['profile.firstName', 'profile.lastName', 'days_since_created']]

yes, it starts with df- displaying
df_no_login[['profile.firstName', 'profile.lastName', 'days_since_created']]
modified_line 
display(df_no_login[['profile.firstName', 'profile.lastName', 'days_since_created']])



<string>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,profile.firstName,profile.lastName,days_since_created
0,Mee,Choi,7
